In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Notebook magic

In [2]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
import subprocess
import os

In [3]:
@magics_class
class PyboardMagic(Magics):
    @cell_magic
    @magic_arguments()
    @argument('-skip')
    @argument('-unix')
    @argument('-pyboard')
    @argument('-file')
    @argument('-data')
    @argument('-time')
    @argument('-memory')
    def micropython(self, line='', cell=None):
        args = parse_argstring(self.micropython, line)
        if args.skip: # doesn't care about the cell's content
            print('skipped execution')
            return None # do not parse the rest
        if args.unix: # tests the code on the unix port. Note that this works on unix only
            with open('/dev/shm/micropython.py', 'w') as fout:
                fout.write(cell)
            proc = subprocess.Popen(["../micropython/ports/unix/build-2/micropython-2", "/dev/shm/micropython.py"], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(proc.stdout.read().decode("utf-8"))
            print(proc.stderr.read().decode("utf-8"))
            return None
        if args.file: # can be used to copy the cell content onto the pyboard's flash
            spaces = "    "
            try:
                with open(args.file, 'w') as fout:
                    fout.write(cell.replace('\t', spaces))
                    printf('written cell to {}'.format(args.file))
            except:
                print('Failed to write to disc!')
            return None # do not parse the rest
        if args.data: # can be used to load data from the pyboard directly into kernel space
            message = pyb.exec(cell)
            if len(message) == 0:
                print('pyboard >>>')
            else:
                print(message.decode('utf-8'))
                # register new variable in user namespace
                self.shell.user_ns[args.data] = string_to_matrix(message.decode("utf-8"))
        
        if args.time: # measures the time of executions
            pyb.exec('import utime')
            message = pyb.exec('t = utime.ticks_us()\n' + cell + '\ndelta = utime.ticks_diff(utime.ticks_us(), t)' + 
                               "\nprint('execution time: {:d} us'.format(delta))")
            print(message.decode('utf-8'))
        
        if args.memory: # prints out memory information 
            message = pyb.exec('from micropython import mem_info\nprint(mem_info())\n')
            print("memory before execution:\n========================\n", message.decode('utf-8'))
            message = pyb.exec(cell)
            print(">>> ", message.decode('utf-8'))
            message = pyb.exec('print(mem_info())')
            print("memory after execution:\n========================\n", message.decode('utf-8'))

        if args.pyboard:
            message = pyb.exec(cell)
            print(message.decode('utf-8'))

ip = get_ipython()
ip.register_magics(PyboardMagic)

## pyboard

In [57]:
import pyboard
pyb = pyboard.Pyboard('/dev/ttyACM0')
pyb.enter_raw_repl()

In [9]:
pyb.exit_raw_repl()
pyb.close()

In [58]:
%%micropython -pyboard 1

import utime
import ulab as np

def timeit(n=1000):
    def wrapper(f, *args, **kwargs):
        func_name = str(f).split(' ')[1]
        def new_func(*args, **kwargs):
            run_times = np.zeros(n, dtype=np.uint16)
            for i in range(n):
                t = utime.ticks_us()
                result = f(*args, **kwargs)
                run_times[i] = utime.ticks_diff(utime.ticks_us(), t)
            print('{}() execution times based on {} cycles'.format(func_name, n, (delta2-delta1)/n))
            print('\tbest: %d us'%np.min(run_times))
            print('\tworst: %d us'%np.max(run_times))
            print('\taverage: %d us'%np.mean(run_times))
            print('\tdeviation: +/-%.3f us'%np.std(run_times))            
            return result
        return new_func
    return wrapper

def timeit(f, *args, **kwargs):
    func_name = str(f).split(' ')[1]
    def new_func(*args, **kwargs):
        t = utime.ticks_us()
        result = f(*args, **kwargs)
        print('execution time: ', utime.ticks_diff(utime.ticks_us(), t), ' us')
        return result
    return new_func

__END_OF_DEFS__

# numpy.random

Random numbers drawn specific distributions can be generated by instantiating a `Generator` object, and calling its methods. The module defines the following three functions:

1. [numpy.random.Generator.normal](#normal)
1. [numpy.random.Generator.random](#random)
1. [numpy.random.Generator.uniform](#uniform)


The `Generator` object, when instantiated, takes a single integer as its argument. This integer is the seed, which will be fed to the 32-bit or 64-bit routine. More details can be found under https://www.pcg-random.org/index.html. The generator is a standard `python` object that keeps track of its state.

`numpy`: https://numpy.org/doc/stable/reference/random/index.html

## normal

A random set of number from the `normal` distribution can be generated by calling the generator's `normal` method. The method takes three optional arguments, `loc=0.0`, the centre of the distribution, `scale=1.0`, the width of the distribution, and `size=None`, a tuple containing the shape of the returned array. In case `size` is `None`, a single floating point number is returned.

The `normal` method of the `Generator` object is based on the [Box-Muller transform](https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform).

`numpy`: https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.normal.html

In [7]:
%%micropython -unix 1

from ulab import numpy as np

rng = np.random.Generator(123456)
print(rng)

# return single number from a distribution of scale 1, and location 0
print(rng.normal())

print(rng.normal(loc=20.0, scale=10.0, size=(3,3)))
# same as above, with positional arguments
print(rng.normal(20.0, 10.0, (3,3)))

Gnerator() at 0x7fa9dae05340
-6.285246229407202
array([[24.95816273705659, 15.2670302229426, 14.81001577336041],
       [20.17589833056986, 23.14539083787544, 26.37772041367461],
       [41.94894234387275, 37.11027030608206, 25.65889562100477]], dtype=float64)
array([[21.52562779033434, 12.74685887865834, 24.08404670765186],
       [4.728112596365396, 7.667757906857082, 21.61576094228444],
       [2.432338873595267, 27.75945683572574, 5.730827584659245]], dtype=float64)




## random

A random set of number from the uniform distribution in the interval [0, 1] can be generated by calling the generator's `random` method. The method takes two optional arguments, `size=None`, a tuple containing the shape of the returned array, and `out`. In case `size` is `None`, a single floating point number is returned. 

`out` can be used, if a floating point array is available. An exception will be raised, if the array is not of `float` `dtype`, or if both `size` and `out` are supplied, and there is a conflict in their shapes.

If `size` is `None`, a single floating point number will be returned.


`numpy`: https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.random.html

In [15]:
%%micropython -unix 1

from ulab import numpy as np

rng = np.random.Generator(123456)
print(rng)

# returning new objects
print(rng.random())
print('\n', rng.random(size=(3,3)))

# supplying a buffer
a = np.array(range(9), dtype=np.float).reshape((3,3))
print('\nbuffer array before:\n', a)
rng.random(out=a)
print('\nbuffer array after:\n', a)

Gnerator() at 0x7f299de05340
6.384615058863119e-11

 array([[0.4348157846574171, 0.7906325931024071, 0.878697619856133],
       [0.8738606263361598, 0.4946080034142021, 0.7765890156101152],
       [0.1770783715717074, 0.02080447648492112, 0.1053837559005948]], dtype=float64)

buffer array before:
 array([[0.0, 1.0, 2.0],
       [3.0, 4.0, 5.0],
       [6.0, 7.0, 8.0]], dtype=float64)

buffer array after:
 array([[0.8508024287393201, 0.9848489829156055, 0.7598167589604003],
       [0.782995698302952, 0.2866337782847831, 0.7915884498022229],
       [0.4614071706315902, 0.4792657443088592, 0.1581582066230718]], dtype=float64)




## uniform

`uniform` is similar to `random`, except that the interval over which the numbers are distributed can be specified, while the `out` argument cannot. In addition to `size` specifying the shape of the output, `low=0.0`, and `high=1.0` are accepted arguments. With the indicated defaults, `uniform` is identical to `random`, which can be seen from the fact that the first 3-by-3 tensor below is the same as the one produced by `rng.random(size=(3,3))` above.


If `size` is `None`, a single floating point number will be returned.


`numpy`: https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.uniform.html

In [14]:
%%micropython -unix 1

from ulab import numpy as np

rng = np.random.Generator(123456)
print(rng)

print(rng.uniform())
# returning numbers between 0, and 1
print('\n', rng.uniform(size=(3,3)))

# returning numbers between 10, and 20
print('\n', rng.uniform(low=10, high=20, size=(3,3)))

# same as above, without the keywords
print('\n', rng.uniform(10, 20, (3,3)))

Gnerator() at 0x7f1891205340
6.384615058863119e-11

 array([[0.4348157846574171, 0.7906325931024071, 0.878697619856133],
       [0.8738606263361598, 0.4946080034142021, 0.7765890156101152],
       [0.1770783715717074, 0.02080447648492112, 0.1053837559005948]], dtype=float64)

 array([[18.5080242873932, 19.84848982915605, 17.598167589604],
       [17.82995698302952, 12.86633778284783, 17.91588449802223],
       [14.6140717063159, 14.79265744308859, 11.58158206623072]], dtype=float64)

 array([[14.3380400319162, 12.72487657409978, 15.77119643621117],
       [13.61835831436355, 18.96062889255558, 15.78847796795966],
       [12.59435855187034, 17.68262037443622, 14.77943040598734]], dtype=float64)


